# 🧹 Fase 2: Limpieza y Preparación de Datos
---  
**🎯 Objetivo:** Transformar los datos crudos en un dataset estructurado y confiable. Nos enfocaremos en la consistencia de tipos de datos, manejo de valores ausentes y preparación de llaves para el Modelo en Estrella (Power BI).

## 1. Importación de Librerías y Carga
> * **Enfoque**: Centralización de dependencias (pandas, numpy) y carga de los 5 datasets validados.
> * **Concepto clave: Data Sourcing**. Es el punto de partida donde aseguramos que la materia prima es la correcta antes de la transformación.

In [3]:
import pandas as pd

In [33]:
customers_raw = pd.read_csv('../data/raw/olist_customers_dataset.csv')
orders_raw = pd.read_csv('../data/raw/olist_orders_dataset.csv')
items_raw = pd.read_csv('../data/raw/olist_order_items_dataset.csv')
products_raw = pd.read_csv('../data/raw/olist_products_dataset.csv')
payments_raw = pd.read_csv('../data/raw/olist_order_payments_dataset.csv')

In [9]:
orders_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [6]:
items_raw.info()
items_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [7]:
products_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [8]:
payments_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [21]:
customers_raw.info()
customers_raw.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


## 2. Transformación de Fechas (Datetime)
> * **Problema:** Las fechas se cargaron como `object` (texto), lo que impide operaciones aritméticas.  
> * **Solución:** Para realizar análisis de series de tiempo y calcular el SLA (Service Level Agreement), convertiré las columnas de texto a formato fecha, estas columnas texto son `orders` y `order_items`.
> * **Terminología: Temporal Data Parsing.** Esto es vital para el Time Intelligence en Power BI; sin esto, no puedo crear calendarios ni medir tendencias mensuales.

In [10]:
# Voy a crear 3 dataframes de trabajo (copias de trabajo) después de cargar los archivos originales:
orders = orders_raw.copy()
items = items_raw.copy()
products = products_raw.copy()

In [35]:
# este me faltó
customers = customers_raw.copy()

In [11]:
# Lista de columnas para `orders`:
columns_fechas_order = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for column in columns_fechas_order:
    orders[column] = pd.to_datetime(orders[column])
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [12]:
# Lista de columnas para `items`
# Aquí solo hay una fecha suave: el límite de envío (shipping_limit_date)
items['shipping_limit_date'] = pd.to_datetime(items['shipping_limit_date'])
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


## 3. Manejo de Valores Faltantes (Nulos)
> * **Estrategia:** Data Imputation vs. Removal.
> * **Nota Técnica:** Documentaré que los nulos en order_delivered_customer_date no siempre son errores, sino órdenes en tránsito o canceladas. No los borraré sin criterio, los aislaré para no ensuciar el cálculo del SLA de órdenes entregadas.

In [14]:
# Conteo de valores nulos por columna en la tabla `orders`
print("Valores nulos en la tabla `orders`")
print(orders.isnull().sum())

Valores nulos en la tabla `orders`
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64


In [15]:
# Ver los valores de orders_status
orders['order_status'].value_counts()

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

In [16]:
# Voy a filtrar los pedidos que NO tienen fecha de entrega y veré su status
nulls_delivered = orders[orders['order_delivered_customer_date'].isnull()]
print("Estado de los pedidos sin fecha de entrega: ")
nulls_delivered['order_status'].value_counts()

Estado de los pedidos sin fecha de entrega: 


order_status
shipped        1107
canceled        619
unavailable     609
invoiced        314
processing      301
delivered         8
created           5
approved          2
Name: count, dtype: int64

In [17]:
# Ver nulos en `products`
print("Nulos en la tabla `products`: ")
print(products.isnull().sum())

Nulos en la tabla `products`: 
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64


In [18]:
# 1. Llenar categorías nulas con 'unknown'
products['product_category_name'] = products['product_category_name'].fillna('unknown')
# 2. Llenar medidas físicas con la mediana (opcional) o 0
# Como solo son 2 registros con peso/dimensiones nulas, no afectarán el análisis.
cols_dimensiones = ['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']
for col in cols_dimensiones:
    products[col] = products[col].fillna(0)

# 3. Llenar conteos con 0
cols_counts = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty']
for col in cols_counts:
    products[col] = products[col].fillna(0)

print("¡Limpieza de nulos en products terminada!")
print(products.isnull().sum())

¡Limpieza de nulos en products terminada!
product_id                    0
product_category_name         0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_length_cm             0
product_height_cm             0
product_width_cm              0
dtype: int64


In [37]:
# 1. Verificación de nulos en `customers_raw`
print(f'Nulos en customers: \n{customers.isnull().sum()}')

Nulos en customers: 
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64


In [39]:
# 2. Validación de unicidad: Cada customer_id debe ser único
duplicates = customers['customer_id'].duplicated().sum()
print(f"Duplicados encontrados: {duplicates}")

Duplicados encontrados: 0


In [41]:
# 3. Estandarización: Asegurar que los estados estén en mayúsculas (por si acaso)
customers['customer_state'] = customers['customer_state'].str.upper()

## 4. Validación de Unicidad y Duplicados
> * **Proceso:** Entity Integrity Validation.
> * **Objetivo:** Verificar la unicidad de las Primary Keys (PK). Un duplicado en order_id duplicaría artificialmente tus ingresos, un error fatal en finanzas.

In [20]:
# Verifico duplicados totales en las tablas de trabajo
print(f"Duplicados en 'orders': {orders.duplicated().sum()}")
print(f"Duplicados en 'items': {items.duplicated().sum()}")
print(f"Duplicados en 'products': {products.duplicated().sum()}")

Duplicados en 'orders': 0
Duplicados en 'items': 0
Duplicados en 'products': 0


In [21]:
# Al no haber duplicados, tengo la certeza de que cada order_id y cada product_id representa una única entidad real

## 5. Integridad de Datos Financieros
* **Objetivo:** Asegurar que los valores de `price`, `freight_value` y `payment_value` sean numéricos y consistentes.
* **Inspección:** Búsqueda de valores atípicos (precios negativos o cero).

In [23]:
# Paso 1: Inspección Estadística
# Estadísticas rápidas de las columnas de dinero
print("Resumen de precios y fletes en 'items':")
print(items[['price','freight_value']].describe())

Resumen de precios y fletes en 'items':
               price  freight_value
count  112650.000000  112650.000000
mean      120.653739      19.990320
std       183.633928      15.806405
min         0.850000       0.000000
25%        39.900000      13.080000
50%        74.990000      16.260000
75%       134.900000      21.150000
max      6735.000000     409.680000


#### Análisis de Integridad Financiera
* **Precios:** Rango entre 0.85 (min) y 6,735.00 (max). No se detectan valores negativos.
* **Fletes:** Rango entre 0.00 y 409.68. Los valores extremos son coherentes con la diversidad de productos y dimensiones geográficas de Brasil.
* **Consistencia:** Los tipos de datos son correctos para cálculos matemáticos.

In [66]:
# Paso 2: Consolidación de Totales (Mentalidad SQL)

# 1. Agrupo los ítems por 'order_id' (Usemos el nombre sin la 's' para que sea más estándar)
order_items_totals = items.groupby('order_id')[['price','freight_value']].sum().reset_index()

# 2. Renombro columnas
order_items_totals.columns = ['order_id', 'total_items_price', 'total_items_freight']

# 3. Calculo el valor final
order_items_totals['order_grand_total'] = order_items_totals['total_items_price'] + order_items_totals['total_items_freight']

print("Resumen de pedidos consolidados (Primeros 5):")
order_items_totals.head()

Resumen de pedidos consolidados (Primeros 5):


,order_id,total_items_price,total_items_freight,order_grand_total
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14,218.04


## 6. Ingeniería de Características (SLA y Logística)
* **Objetivo:** Crear métricas que permitan medir la eficiencia del servicio de entrega.
* **Métricas creadas:**
    * `delivery_time_days`: Tiempo real que tomó la entrega (Fecha entrega - Fecha compra).
    * `delta_estimated_days`: Diferencia entre la fecha de entrega real y la fecha estimada (Retrasos o entregas anticipadas).

In [27]:
orders.head()
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [28]:
# Calculo el tiempo real de entrega
orders['delivery_time_days'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days
# 2. Calculo la diferencia contra la promesa (SLA)
# Negativo = 'Antes' de tiempo (Bueno) | Positivo = Retraso (Malo)
orders['delta_estimated_days'] = (orders['order_delivered_customer_date'] - orders['order_estimated_delivery_date']).dt.days

In [29]:
print("Métricas de SLA calculadas exitosamente.")
print(orders[['delivery_time_days', 'delta_estimated_days']].describe())

Métricas de SLA calculadas exitosamente.
       delivery_time_days  delta_estimated_days
count        96476.000000          96476.000000
mean            12.094086            -11.876881
std              9.551746             10.183854
min              0.000000           -147.000000
25%              6.000000            -17.000000
50%             10.000000            -12.000000
75%             15.000000             -7.000000
max            209.000000            188.000000


#### 📝 Análisis de Métricas Logísticas (SLA)
* **Eficiencia de Entrega (delivery_time_days)**: El tiempo promedio de entrega es de 12 días, con una mediana de 10 días. La diferencia sugiere la presencia de pedidos con tiempos de entrega inusualmente largos.
* **Cumplimiento de Promesa (delta_estimated_days)**: En promedio, los productos llegan 11.8 días antes de la fecha estimada. Esto indica una estrategia de Olist de sobreestimar los tiempos de entrega para mejorar la satisfacción del cliente.
* **Casos Atípicos (Outliers)**: Se detectaron casos extremos con hasta 209 días de entrega y retrasos de 188 días sobre la promesa inicial. Estos registros se mantendrán para un análisis profundo en la etapa de logística (Notebook 05).

In [57]:
# Modo repaso: Modife orders para las fechas y nulos, items para fechas y productos para nulos.

## 7. Exportación de Datos Procesados
* **Objetivo:** Guardar los datasets limpios y transformados en la carpeta `processed` para su uso en los notebooks de análisis.

In [68]:
import os

path_processed = '../data/processed'

# Crea la carpeta (y las intermedias) si NO existen.
# Si ya existen, NO lanza error gracias a exist_ok=True.
os.makedirs(path_processed, exist_ok=True)

# Exporto los DataFrames a CSV.
# index=False evita que se exporte la columna del índice del DataFrame.
orders.to_csv(f"{path_processed}/orders_cleaned.csv", index=False)
items.to_csv(f"{path_processed}/items_cleaned.csv", index=False)
products.to_csv(f"{path_processed}/products_cleaned.csv", index=False)
order_items_totals.to_csv(f"{path_processed}/order_items_totals.csv", index=False)

print("¡Proceso completado! Los 4 datasets están listos en la carpeta /data/processed/")

¡Proceso completado! Los 4 datasets están listos en la carpeta /data/processed/


In [ ]:
# Exporto lo que faltó

In [43]:
import os 
customers.to_csv("../data/processed/customers_cleaned.csv", index=False)